In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install sentence_transformers
!pip install lime

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import string

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
from tqdm import tqdm
from operator import itemgetter

In [ ]:
def load_data(data_file):
  # read csv file
  df = pd.read_csv(data_file)

  # replace nan(no value) comment with ""(empty string)
  df.fillna("", inplace=True)

  comments = df['comments'].tolist()
  genders = df['labels'].tolist()

  genders = [0 if gender == "Male" else 1 for gender in genders]

  return comments, genders

In [ ]:
comments, genders = load_data('/content/drive/MyDrive/Datasets/Bert Modal On One Comment Full Data/cleaned_dataset.csv')

# df = pd.DataFrame(list(zip(comments, genders)),
#                columns =['comments', 'genders'])

# print(df)

comments = np.array(comments)
genders = np.array(genders)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(comments, genders,
                                                    test_size=0.2,
                                                    stratify=genders,
                                                    random_state=42)
print(len(X_train),len( X_test), len(y_train),len( y_test))

In [ ]:
# Model

top_words = 300000
max_comment_length = 300
embedding_vecor_length = 64

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
list_tokenized_train = tokenizer.texts_to_sequences(X_train)

In [ ]:
X_train = pad_sequences(list_tokenized_train, maxlen=max_comment_length)

In [ ]:
model = Sequential()
model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_comment_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train,y_train, epochs=10, batch_size=64)

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(list_tokenized_test, maxlen=max_comment_length)
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)


In [ ]:
def load_data_for_replaced_dataset(data_file):
  # read csv file
  df = pd.read_csv(data_file)

  # replace nan(no value) comment with ""(empty string)
  df.fillna("", inplace=True)

  comments = df['comment'].tolist()
  genders = df['user_gender'].tolist()

  genders = [0 if gender == "Male" else 1 for gender in genders]

  return comments, genders

In [ ]:
replaced_comments, replaced_genders = load_data_for_replaced_dataset('/content/drive/MyDrive/Datasets/RNN/combined_replaced_dataset_one_comment_rnn_with_changed.csv')

replaced_comments = np.array(replaced_comments)
replaced_genders = np.array(replaced_genders)

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(replaced_comments)
changed_replaced_comments = pad_sequences(list_tokenized_test, maxlen=max_comment_length)
prediction = model.predict(changed_replaced_comments)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, replaced_genders))
print('F1-score: ', f1_score(y_pred, replaced_genders))
print('Confusion matrix:')
confusion_matrix(replaced_genders,y_pred)
